In this series of notebooks, I will attempt to do some introductory exploration of various roller derby statistics. We will use the publicly available stats on the FlatTrackStats website. First, I will build a table scraper tool using the BeautifulSoup4 package to parse the stats tables on the website. If not already installed, you will need pandas and BeautifulSoup4 in order to run this notebook. 

In [12]:
    import requests
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from itertools import product
    import urllib.request
    import urllib.parse
    
    import collections
    import re
    
    #First, define a class to parse HTML tables for bouts and players
    
    class HTMLTableParser:
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.read_table(table)) for table in soup.find_all('table')]      
        
        def read_table(self, table_tag):
            rowspans = []  # track pending rowspans
            rows = table_tag.find_all('tr')

            # first scan, see how many columns we need
            colcount = 0
            column_names=[]
            for r, row in enumerate(rows):
                cells = row.find_all(['td', 'th'], recursive=False)
                colcount = max(
                    colcount,
                    sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
                # update rowspan bookkeeping; 0 is a span to the bottom. 
                rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
                rowspans = [s - 1 for s in rowspans if s > 1]

            # it doesn't matter if there are still rowspan numbers 'active'; no extra
            # rows to show in the table means the larger than 1 rowspan numbers in the
            # last table row are ignored.    
            # build an empty matrix for all possible cells
            table = [[None] * colcount for row in rows]


            # fill matrix from row data
            rowspans = {}  # track pending rowspans, column number mapping to count
            for row, row_elem in enumerate(rows):
                span_offset = 0  # how many columns are skipped due to row and colspans 
                for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
                    # adjust for preceding row and colspans
                    col += span_offset
                    while rowspans.get(col, 0):
                        span_offset += 1
                        col += 1

                    # fill table data
                    rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
                    colspan = int(cell.get('colspan', 1)) or colcount - col
                    # next column is offset by the colspan
                    span_offset += colspan - 1
                    value = cell.get_text()
                    points = len(cell.find_all("div", {"class": " point"}))
                    pens = cell.find_all("div", {"class": "penalty major"})  
                    if (value == '' or value == '&nbsp') and points != '':
                        value = str(points)
                    if len(pens) > 0:
                        value = ''
                        for pen in pens:
                            value += pen.find_all(text=True, recursive=False)[0] + " "
                    if len(cell.find_all("div", {"class": "lead"}))== 1:
                        if value == "0": value = ""
                        value += "Lead "
                    if len(cell.find_all("div", {"class": "leadloss"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "
                    if len(cell.find_all("div", {"class": "lost"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "                   
                    if len(cell.find_all("div", {"class": "call"}))== 1:
                        if value == "0": value = ""
                        value += "call "
                    if len(cell.find_all("div", {"class": "nopass"}))== 1:
                        if value == "0": value = ""
                        value += "np "    
                    if len(cell.find_all("div", {"class": "starpass"}))== 1:
                        if value == "0": value = ""
                        value += "sp "    
                    for drow, dcol in product(range(rowspan), range(colspan)):
                        try:
                            table[row + drow][col + dcol] = value
                            rowspans[col + dcol] = rowspan
                        except IndexError:
                            # rowspan or colspan outside the confines of the table
                            pass

                # update rowspan bookkeeping
                rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}
            npt = np.array(table)
            #df = pd.DataFrame(np.array(table), column_names) 
            return table

        def parse_url_todf(self, url):
            tables = self.parse_url(url)
            dfs = []
            if "combos" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "teams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "jams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)        
            else:
                headers = tables[0]
                headersnew = []
                for header in headers:
                    headernew = [i for i in header if i] 
                    headersnew.append(headernew)
                for i in range(len(tables)-1):
                    df = pd.DataFrame(np.array(tables[i+1]), columns=headersnew[i])    
                    dfs.append(df)
                
            return dfs    

Let's make a function to get all of a team's bouts from the database:

In [13]:
def GetAllBouts(teamID):
    npages = 0
    links=[]
    
    base_url = "http://www.flattrackstats.com/teams/"+teamID+"/bouts"
    text = urllib.request.urlopen(base_url)
    base_soup = BeautifulSoup(text)
    
    for listitem in base_soup.findAll('li', class_="pager-last last"):
        npages = int(listitem.findAll('a')[0]['href'][-1])
    
    
    for page in range(0,npages):
        url = "http://www.flattrackstats.com/teams/"+teamID+"/bouts?page="+str(page)
        text = urllib.request.urlopen(url)
        soup = BeautifulSoup(text)
        for link in soup.findAll('a', class_="boutlink has-stats"):
            linkname = link['href']
            linksize = len(linkname)
            if "node" not in link['href']:
                linkstub = linkname[:linksize-8]
            else: linkstub = "/bouts"+linkname[5:]+"/"
            links.append(linkstub)
                
    return links

In [14]:
GetAllBouts(str(3637))

['/bouts/89490/',
 '/bouts/89482/',
 '/bouts/76964/',
 '/bouts/64102/',
 '/bouts/59173/',
 '/bouts/55689/',
 '/bouts/55648/',
 '/bouts/53728/',
 '/bouts/45726/',
 '/bouts/47480/',
 '/bouts/42867/',
 '/bouts/37439/',
 '/bouts/37415/',
 '/bouts/35520/',
 '/bouts/35871/',
 '/bouts/30536/',
 '/bouts/27122/',
 '/bouts/20699/',
 '/bouts/8252/',
 '/bouts/20350/',
 '/bouts/22251/',
 '/bouts/20353/',
 '/bouts/4326/',
 '/bouts/19804/',
 '/bouts/3930/']

Now let's make some functions to read the data from the bouts page (one such page is here: http://flattrackstats.com/bouts/55689/jams).

The first will be a function to read the table of points scored in each jam, logged per lap. This is similar in format to what is recorded on the track. Let's make sure to drop the opposing team's scores from this.

In [15]:
def GetAllScores(teamID, teamName):
    scoreframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        scores = hp.parse_url_todf(jampage)[1].iloc[:,1:-1]
        scores.loc[np.c_[scores.index[1::2].tolist(),scores.index[0:-1:2].tolist()].reshape(-1)]
        
        #del scores[0]
        #print(scores)
        scoreframes.append(scores)
    allScores = pd.concat(scoreframes)
    #drop opposing team
    allScores = allScores.iloc[::2]
    allScores = allScores.reset_index()
    del allScores['index']
    #print(allScores)
    return(allScores)

Next, let's make a function to get the table recording who played in each jam, and what the cumulative score, jam score, and score differential (my_team - opposing_team) was for that jam. Again, let's drop the opposing team's lineups. This is the function that will give us most of our data!

In [43]:
def GetAllLineups(teamID, teamName):
    lineupframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True        
        lineups = hp.parse_url_todf(jampage)[2].iloc[:,1:-1]
        
        #put team of interest first (data is naturally in format (home, away) )
        if isAway:
            lineups = lineups.loc[np.c_[lineups.index[1::2].tolist(),lineups.index[0:-1:2].tolist()].reshape(-1)]
        lineupframes.append(lineups) 
        
    #clean data    
    allLineups = pd.concat(lineupframes)
    allLineups.columns = ['Jammer','Jstats','B1','B2','B3','B4','NULL','jamscore','runscore']
    del allLineups['NULL']
    
    allLineups = allLineups[~(allLineups.jamscore == 'Period 2')]
    allLineups = allLineups[~(allLineups.jamscore.isnull())]
    
    #calc score differentials, for determining rankings
    allLineups['jamscore'] = pd.to_numeric(allLineups['jamscore'])
    allLineups['runscore'] = pd.to_numeric(allLineups['runscore'])
    
    allLineups['ScoreDiff'] = allLineups['jamscore'].diff(periods=-1)
    
    #drop opposing team
    allLineups = allLineups.iloc[::2]
    allLineups = allLineups.reset_index()
    del allLineups['index']
    
    return(allLineups)

Next, let's make a function to get the penalties per jam. This will need to be cross-referenced with the lineups table in order to determine who got what penalties.

In [44]:
def GetAllPenalties(teamID, teamName):
    penframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        pens = hp.parse_url_todf(jampage)[3].iloc[:,1:-1]
        
        if isAway:
            pens.loc[np.c_[pens.index[1::2].tolist(),pens.index[0:-1:2].tolist()].reshape(-1)]

        #del pens[0]
        penframes.append(scores)
    allPens = pd.concat(penframes)
    #print(allPens)
    return(allPens)

In lieu of rankings, let's use an average of the score differential for each blocker. Let's get an expanding average of the blocker's point differential (that is, after a given jam, the point differential for each jam the blocker has played in, averaged over all the times they have played).

For the time being, we'll assume we don't need a certain number of jams played in for the average to stabilize. This is likely not correct (fewer data points == more sensitivity to noise!), but we don't have many stats for each individual player.

In [45]:
def ExpandingAverages(teamID, teamName):
    lineups = GetAllLineups(teamID,teamName)
    allBlockers = (lineups['B1'].append(lineups['B2']).append(lineups['B3']).append(lineups['B4'])).unique()
    #print(allBlockers, len(allBlockers))
    blockerCols = lineups[['B1','B2','B3','B4']]
    #print(blockerCols)
    blockerlist = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis = 1)
    indicators = blockerlist.groupby(by=blockerlist.columns, axis=1).sum()
    scorediffs = indicators.mul(lineups['ScoreDiff'], axis=0)
    scorediffs.replace(0, np.nan, inplace=True)
    runningmean = scorediffs.expanding(1).mean()
    runningmean.replace(np.nan, 0, inplace=True)
    pd.set_option('display.max_columns', 58)  # or 1000
    pd.set_option('display.max_rows', 20)  # or 1000
    pd.set_option('display.max_colwidth', 15)  # or 199
    #test2 = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis=1)
    #test = lineups.stack().str.get_dummies(columns =['B1','B2','B3','B4']).sum(level=0)
    return(runningmean)

In [46]:
def AvgChangeFromJam(teamID, teamName):
    avgs = ExpandingAverages(teamID, teamName)
    avgswzero = pd.DataFrame([[0.0] * len(avgs.columns)], columns=avgs.columns)
    avgs = avgswzero.append(avgs, ignore_index=True)
    deltas = avgs.diff()
    deltas = deltas.drop([0])
    return deltas

In [47]:
AvgChangeFromJam(str(3637),'Killamazoo')

,???,Afterburner,Aly-Kate Contusion...,Anita Tuf Moth'r,BAM,Badass Maggie,Battleaxe,Beaver Jam,Belles,Beverly Hells,Bicked Witch,Buns N Roses,Charisnakov,Crashive Aggressiv...,Delilah Danger,Doobie Trap,DuBois,Emma Pistol,Goin Commando,Heartless Hottie,Hill-De-Beast,Ivanna O'Bliviacha,Jane Deere,Javelin,Jessie Girl,Killamity Cate,Lady Hawk,Lem E. Atom,Lily St. Smear,Maggie Walters,Mea Chokeavich,Merica,Miss Andry,Miss B. Haven (KDD...,Miss Bea Haven (PO...,Mustang,Neva,Noam Stompsky,Ophelia Plenty,Painbow Connection,Protasiewicz,Ramona D. Flowers ...,RocKickAss,Rocket HIps,Rogue V Wade,Rolla Ramher,Rosie Ferocious,Smash Bandicute,Smash UrAsh,Sparkills (KDD),Spicy Jalapeno,Stinker Belle,Terrorhawk,The Heckler,Trouble Chicken,Weers Waldo,Wreck Keene Ball,Yoga Nagettit
1,0.0,0.0,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-4.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,-7.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.500000,0.0,0.0,0.000000,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,-3.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1065,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1066,0.0,0.0,0.000000,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-0.660714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.121341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.151568
1067,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [48]:
GetAllLineups(str(3637),'Killamazoo')

,Jammer,Jstats,B1,B2,B3,B4,jamscore,runscore,ScoreDiff
0,Beaver Jam,Lead,Aly-Kate Co...,Wreck Keene...,Mustang,Javelin,10,10,10.0
1,Buns N Roses,0,Painbow Con...,Smash Bandi...,Maggie Walters,Ophelia Plenty,0,10,-4.0
2,Weers Waldo,0,Painbow Con...,Aly-Kate Co...,Mea Chokeavich,Javelin,0,10,-5.0
3,Hill-De-Beast,0,Smash Bandi...,Beverly Hells,Maggie Walters,Ophelia Plenty,0,10,-4.0
4,Beaver Jam,0,Smash Bandi...,Wreck Keene...,Aly-Kate Co...,Javelin,0,10,-7.0
...,...,...,...,...,...,...,...,...,...
1063,Beverly Hells,0,Sparkills (...,Rosie Feroc...,Aly-Kate Co...,Ramona D. F...,4,35,0.0
1064,Delilah Danger,Lead,Javelin,Ivanna O'Bl...,Killamity Cate,Lily St. Smear,0,35,0.0
1065,Rosie Feroc...,0,Lady Hawk,Noam Stompsky,Battleaxe,Yoga Nagettit,0,35,-9.0
1066,Sparkills (...,LeadLoss,Ivanna O'Bl...,Ramona D. F...,Aly-Kate Co...,Lily St. Smear,0,35,0.0
